In [1]:
import re
import os

In [70]:
fpath = 'E:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718_GW2A18/source/Drv_IF.v'
npath = 'E:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718_GW2A18/source/Drv_IF_Pro.v'
wrpath = 'E:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718_GW2A18/source/wires.v'

In [11]:
raw_data = ''
with open(fpath,'r') as f:
    raw_data = f.readlines()

In [12]:
raw_data

['`timescale 1ns / 1ps\n',
 '//////////////////////////////////////////////////////////////////////////////////\n',
 '// Company: \n',
 '// Engineer: \n',
 '// \n',
 '// Create Date:    08:45:25 11/10/2017 \n',
 '// Design Name: \n',
 '// Module Name:    Drv_IF \n',
 '// Project Name: \n',
 '// Target Devices: \n',
 '// Tool versions: \n',
 '// Description: \n',
 '//\n',
 '// Dependencies: \n',
 '//\n',
 '// Revision: \n',
 '// Revision 0.01 - File Created\n',
 '// Additional Comments: \n',
 '//\n',
 '// C2.5 Panel: 192x108; IM: 96x36; Input / IM: 3; section / input: 2, 32x18\n',
 '// scan: 18\n',
 '// LED error detection: not support\n',
 '// LED driver IC: CHIPONE\n',
 '//\n',
 '// -------------------------------------------------------\n',
 '// | \t  1\t| \t  2\t| \t  3\t| \t  10\t| \t  11\t| \t  12\t|\n',
 '// |\t\t\t|\t\t\t|\t\t\t|\t\t\t|\t\t\t|\t\t\t|\n',
 '// -------------------------------------------------------\n',
 '// | \t  4\t| \t  5\t| \t  6\t| \t  13\t| \t  14\t| \t  15\

In [5]:
m1 = r'if\s?\((.+)\)'
p1 = re.compile(m1)

In [6]:
condtions = p1.findall(raw_data)

In [7]:
condtions

['state != S4',
 '(SOUT_EN == bit_on) && (SPRECK_cnt > DCLKr) && !DCLK',
 'state == S3',
 'Cnt_LAT == LAT_Wr_Reg_1',
 'Cnt_LAT == LAT_Wr_Reg_2',
 'Cnt_LAT == LAT_Wr_Reg_3',
 'Cnt_LAT == LAT_Wr_Reg_4',
 'Cnt_LAT == LAT_Wr_Debug',
 'FRM_ALT == bit_off',
 'RD_TOP == bit_on',
 "DCLK && (Cnt_DCLK == 8'd0) && (SPRECK_cnt == 10'd0) && (Cnt_LAT == LAT_VSYNC)",
 "!DCLK && (SPRECK_cnt == 10'd0) && (wr_reg == LAT_off) && (Cnt_DCLK == 8'd0) && (Cnt_IC == 8'd0)",
 "(Cnt_Pixel_s == Total_Pixel) && LAT && DCLK && (Cnt_DCLK == 8'd0) && (SPRECK_cnt == 10'd0)",
 '(DCLK !== bit_on) && (DCLK !== bit_off)',
 "SPRECK_cnt == 10'd0",
 'state == S3',
 'Cnt_LAT == LAT_off',
 'state == S4',
 "(RD_EN_off < (RD_EN_off_s - 10'd1)) && (RD_EN_off > 10'd0)",
 "(RD_EN && !SOUT_EN) || ((fifo_mem_bit == 4'd15) && !SOUT_EN && (SPRECK_cnt == 10'd0))",
 '(state < S2) || (state > S4)',
 'state == S2',
 "(Cnt_LAT == LAT_VSYNC) && (Cnt_DCLK > 8'd3)",
 '(GCLK !== bit_on) && (GCLK !== bit_off)',
 "(state != S2) && (Cnt_GCLK >= 8

In [65]:
def get_conditions(dlines):
    m = r'if\s?\((.+)\)'
    p = re.compile(m)
    conditions = []
    for line in dlines:
        if not line.strip().startswith('//'):
            r = p.findall(line)
            if r:
                conditions.append(r[0].strip())
    
    excels = {}
    results = {}
    n = 0
    for cdt in conditions:
        sp1 = [x.strip() for x in cdt.split('&&') if x]
        sp2 = []
        if sp1:
            for x in sp1:
                for y in x.split('||'):
                    if y:
                        sp2.append(y.strip())
        sp3 = []
        for x in sp2:
            a = x.count('(')
            b = x.count(')')
            y = x
            if a > b:
                y = x[1:]
            elif a < b:
                y = x[:-1]
            sp3.append(y)
        sp2 = sp3.copy()
        #if cdt not in excels:
        #    excels[cdt] = []
        if cdt not in excels: 
            if len(sp2) == 1:
                z = [x for x in sp2[0].split(' ') if x]
                if len(z) > 1:
                    k = 'cp' + str(n)
                    excels[k] = sp2[0]
                    excels[sp2[0]] = k
                    n+=1
                else:
                    excels[sp2[0]] = sp2[0]
            elif len(sp2) > 1:
                #if cdt not in excels:
                #    excels[cdt] = sp2
                res = cdt
                for sp in sp2:
                    if sp not in excels:
                        z = [x for x in sp.split(' ') if x]
                        if len(z) > 1:
                            k = 'cp' + str(n)
                            excels[k] = sp
                            excels[sp] = k
                            n+=1
                            res = res.replace(sp,k)
                        else:
                            excels[sp] = sp
                    else:
                        res = res.replace(sp,excels[sp])
                #results[cdt] = res
                #excels[cdt] = res
                k = 'cp' + str(n)
                results[cdt] = k
                excels[k] = res
                
                #excels[cdt] = k
                n+=1            
    return excels,results

In [66]:
excels, results = get_conditions(raw_data)

In [67]:
excels

{'cp0': 'state != S4',
 'state != S4': 'cp0',
 'cp1': '(SOUT_EN == bit_on)',
 '(SOUT_EN == bit_on)': 'cp1',
 'cp2': '(SPRECK_cnt > DCLKr)',
 '(SPRECK_cnt > DCLKr)': 'cp2',
 '!DCLK': '!DCLK',
 'cp3': 'cp1 && cp2 && !DCLK',
 'cp4': 'state == S3',
 'state == S3': 'cp4',
 'cp5': 'FRM_ALT == bit_off',
 'FRM_ALT == bit_off': 'cp5',
 'cp6': 'RD_TOP == bit_on',
 'RD_TOP == bit_on': 'cp6',
 'DCLK': 'DCLK',
 'cp7': "(Cnt_DCLK == 8'd0)",
 "(Cnt_DCLK == 8'd0)": 'cp7',
 'cp8': "(SPRECK_cnt == 10'd0)",
 "(SPRECK_cnt == 10'd0)": 'cp8',
 'cp9': '(Cnt_LAT == LAT_VSYNC)',
 '(Cnt_LAT == LAT_VSYNC)': 'cp9',
 'cp10': 'DCLK && cp7 && cp8 && cp9',
 'cp11': '(wr_reg == LAT_off)',
 '(wr_reg == LAT_off)': 'cp11',
 'cp12': "(Cnt_IC == 8'd0)",
 "(Cnt_IC == 8'd0)": 'cp12',
 'cp13': '!DCLK && cp8 && cp11 && cp7 && cp12',
 'cp14': '(Cnt_Pixel_s == Total_Pixel)',
 '(Cnt_Pixel_s == Total_Pixel)': 'cp14',
 'LAT': 'LAT',
 'cp15': 'cp14 && LAT && DCLK && cp7 && cp8',
 'cp16': 'state == S4',
 'state == S4': 'cp16',
 'cp17

In [68]:
results

{'(SOUT_EN == bit_on) && (SPRECK_cnt > DCLKr) && !DCLK': 'cp3',
 "DCLK && (Cnt_DCLK == 8'd0) && (SPRECK_cnt == 10'd0) && (Cnt_LAT == LAT_VSYNC)": 'cp154',
 "!DCLK && (SPRECK_cnt == 10'd0) && (wr_reg == LAT_off) && (Cnt_DCLK == 8'd0) && (Cnt_IC == 8'd0)": 'cp158',
 "(Cnt_Pixel_s == Total_Pixel) && LAT && DCLK && (Cnt_DCLK == 8'd0) && (SPRECK_cnt == 10'd0)": 'cp194',
 "(RD_EN_off < (RD_EN_off_s - 10'd1)) && (RD_EN_off > 10'd0)": 'cp19',
 "(RD_EN && !SOUT_EN) || ((fifo_mem_bit == 4'd15) && !SOUT_EN && (SPRECK_cnt == 10'd0))": 'cp21',
 '(state < S2) || (state > S4)': 'cp24',
 "(Cnt_LAT == LAT_VSYNC) && (Cnt_DCLK > 8'd3)": 'cp27',
 "(state != S2) && (Cnt_GCLK >= 8'd1) && (Cnt_GCLK_off == 5'd0) && (SPRECK_cnt % DCLKr) == 10'd0": 'cp32',
 "(Cnt_GCLK == 8'd0) || (Cnt_GCLK_off > 5'd0)": 'cp35',
 "(Cnt_GCLK >= 8'd1) && (Cnt_GCLK != 8'd0) && (Cnt_GCLK_off == 5'd0) && (SPRECK_cnt % DCLKr) == 10'd0": 'cp38',
 "DCLK && (Cnt_DCLK == 8'd0) && (SPRECK_cnt == 10'd0) && (Cnt_LAT == LAT_off)": 'cp151',
 "

In [73]:
def gen_vlg(combs:{},results:{},dlines, npath=npath, wrpath=wrpath):
    wires = []
    entitys = ''.join(dlines)
    mark = r'cp\d+'
    pat = re.compile(mark)
    for k,v in combs.items():
        if pat.search(k):
            wire = 'wire ' + k + ' = ' + v + ';\n'
            wires.append(wire)
    for k,v in results.items():
        entitys = entitys.replace(k,v)
    
    with open(wrpath,'w') as f:
        f.writelines(wires) 
        
    with open(npath,'w') as f:
        f.write(entitys)        
    return wires, entitys

In [74]:
wires, entitys = gen_vlg(excels, results, raw_data)

In [75]:
wires

['wire cp0 = state != S4;\n',
 'wire cp1 = (SOUT_EN == bit_on);\n',
 'wire cp2 = (SPRECK_cnt > DCLKr);\n',
 'wire cp3 = cp1 && cp2 && !DCLK;\n',
 'wire cp4 = state == S3;\n',
 'wire cp5 = FRM_ALT == bit_off;\n',
 'wire cp6 = RD_TOP == bit_on;\n',
 "wire cp7 = (Cnt_DCLK == 8'd0);\n",
 "wire cp8 = (SPRECK_cnt == 10'd0);\n",
 'wire cp9 = (Cnt_LAT == LAT_VSYNC);\n',
 'wire cp10 = DCLK && cp7 && cp8 && cp9;\n',
 'wire cp11 = (wr_reg == LAT_off);\n',
 "wire cp12 = (Cnt_IC == 8'd0);\n",
 'wire cp13 = !DCLK && cp8 && cp11 && cp7 && cp12;\n',
 'wire cp14 = (Cnt_Pixel_s == Total_Pixel);\n',
 'wire cp15 = cp14 && LAT && DCLK && cp7 && cp8;\n',
 'wire cp16 = state == S4;\n',
 "wire cp17 = (RD_EN_off < (RD_EN_off_s - 10'd1));\n",
 "wire cp18 = (RD_EN_off > 10'd0);\n",
 'wire cp19 = cp17 && cp18;\n',
 "wire cp20 = (fifo_mem_bit == 4'd15);\n",
 'wire cp21 = (RD_EN && !SOUT_EN) || (cp20 && !SOUT_EN && cp8);\n',
 'wire cp22 = (state < S2);\n',
 'wire cp23 = (state > S4);\n',
 'wire cp24 = cp22 || cp23;

In [76]:
entitys

"`timescale 1ns / 1ps\n//////////////////////////////////////////////////////////////////////////////////\n// Company: \n// Engineer: \n// \n// Create Date:    08:45:25 11/10/2017 \n// Design Name: \n// Module Name:    Drv_IF \n// Project Name: \n// Target Devices: \n// Tool versions: \n// Description: \n//\n// Dependencies: \n//\n// Revision: \n// Revision 0.01 - File Created\n// Additional Comments: \n//\n// C2.5 Panel: 192x108; IM: 96x36; Input / IM: 3; section / input: 2, 32x18\n// scan: 18\n// LED error detection: not support\n// LED driver IC: CHIPONE\n//\n// -------------------------------------------------------\n// | \t  1\t| \t  2\t| \t  3\t| \t  10\t| \t  11\t| \t  12\t|\n// |\t\t\t|\t\t\t|\t\t\t|\t\t\t|\t\t\t|\t\t\t|\n// -------------------------------------------------------\n// | \t  4\t| \t  5\t| \t  6\t| \t  13\t| \t  14\t| \t  15\t|\n// |\t\t\t|\t\t\t|\t\t\t|\t\t\t|\t\t\t|\t\t\t|\n// -------------------------------------------------------\n// | \t  7\t| \t  8\t| \t  9\

In [52]:
with open(npath,'w') as f:
    f.write(entitys)

In [ ]:
str.st

In [20]:
a = 'aaabbbccc'

In [21]:
a.replace('aaa','111')

'111bbbccc'

In [22]:
a

'aaabbbccc'

In [31]:
a.count('a')

3

In [27]:
b = "(RD_EN && !SOUT_EN) || ((fifo_mem_bit == 4'd15) && !SOUT_EN && (SPRECK_cnt == 10'd0))"

In [28]:
seg1 = [x.strip() for x in b.split('&&') if x]
seg2 = []
if seg1:
    for x in seg1:
        for y in x.split('||'):
            if y:
                seg2.append(y.strip())

In [29]:
seg2

['(RD_EN',
 '!SOUT_EN)',
 "((fifo_mem_bit == 4'd15)",
 '!SOUT_EN',
 "(SPRECK_cnt == 10'd0))"]

In [30]:
str.count?

In [32]:
seg3 = []
for x in seg2:
    a = x.count('(')
    b = x.count(')')
    y = x
    if a > b:
        y = x[1:]
    elif a < b:
        y = x[:-1]
    seg3.append(y)

In [33]:
seg3

['RD_EN',
 '!SOUT_EN',
 "(fifo_mem_bit == 4'd15)",
 '!SOUT_EN',
 "(SPRECK_cnt == 10'd0)"]